In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CPUFreqUtility',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1803234

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
len(bugzillas)

789

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo") or []
    for needinfo in needinfos:
        if needinfo['requestee'] == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/percona-xtrabackup
rpms/cook
rpms/Pencil2D
rpms/RxCpp
rpms/apiviz
rpms/apper
rpms/arc-theme
rpms/bfast
rpms/biblesync
rpms/bifcl
rpms/bmon
rpms/coolreader
rpms/couchdb
rpms/cpulimit
rpms/dc3dd
rpms/dep
rpms/dnsjava
rpms/dpm-xrootd
rpms/dumb-init
rpms/dynaplugz
rpms/edb
rpms/electric
rpms/erlang-corba
rpms/erlang-lager
rpms/eyesight
rpms/fifechan
rpms/fillets-ng
rpms/flacon
rpms/fluxbox
rpms/fmpp
rpms/fop
rpms/fossil
rpms/freetiger
rpms/gaupol
rpms/golang-contrib-opencensus-exporter-ocagent
rpms/golang-github-casbin
rpms/golang-github-cenkalti-backoff
rpms/golang-github-cespare-xxhash
rpms/glyr
rpms/gmpc
rpms/gnome-code-assistance
rpms/libquvi
rpms/gnome-documents
rpms/powermock
rpms/golang-github-henvic-httpretty
rpms/golang-github-kdar-factorlog
rpms/golang-github-knieriem-markdown
rpms/golang-github-paulbellamy-ratecounter
rpms/golang-github-xorm
rpms/golang-k8s-kubernetes
rpms/goobook
rpms/gplugin
rpms/gridftp-ifce
rpms/hsakmt
rpms/icemon
rpms/innoextract
rpms/jaxodraw
rpms/jbl

rpms/nodejs-jest-mock
rpms/nodejs-jsonfile
rpms/nodejs-jsonparse
rpms/nodejs-jsonpointer-dot-js
rpms/nodejs-json-stable-stringify
rpms/nodejs-json-stringify-safe
rpms/nodejs-js-tokens
rpms/nodejs-keep-alive-agent
rpms/nodejs-keygrip
rpms/nodejs-kind-of
rpms/nodejs-lazy-cache
rpms/nodejs-leche
rpms/nodejs-left-pad
rpms/nodejs-line-numbers
rpms/nodejs-line-reader
rpms/nodejs-linkify-it
rpms/nodejs-locate-character
rpms/nodejs-log-driver
rpms/nodejs-log-ok
rpms/nodejs-log-utils
rpms/nodejs-long
rpms/nodejs-longest
rpms/nodejs-makedir
rpms/nodejs-make-error
rpms/nodejs-map-cache
rpms/nodejs-map-obj
rpms/nodejs-map-visit
rpms/nodejs-markdown
rpms/nodejs-matched
rpms/nodejs-max-timeout
rpms/nodejs-md5
rpms/nodejs-md5-o-matic
rpms/nodejs-mdn-data
rpms/nodejs-mdurl
rpms/nodejs-media-typer
rpms/nodejs-memoize-path
rpms/nodejs-metascript
rpms/nodejs-mime-db
rpms/nodejs-mime-types
rpms/nodejs-minimalistic-assert
rpms/nodejs-minimatch
rpms/nodejs-minimist
rpms/nodejs-mixin-deep
rpms/nodejs-mixin-o

In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1799854,1859835,1863091,1863115,1863215,1863216,1863219,1863220,1863225,1863246,1863248,1863253,1863280,1863363,1863366,1863372,1863401,1863420,1863436,1863449,1863456,1863462,1863472,1863478,1863491,1863507,1863523,1863553,1863555,1863561,1863572,1863574,1863576,1863577,1863587,1863596,1863609,1863638,1863641,1863642,1863643,1863644,1863646,1863661,1863669,1863672,1863704,1863755,1863761,1863762,1863769,1863778,1863798,1863813,1863816,1863831,1863856,1863875,1863886,1863896,1863897,1863900,1863902,1863903,1863905,1863906,1863910,1863911,1863912,1863914,1863915,1863916,1863919,1863921,1863928,1863946,1863950,1863971,1863977,1863993,1863995,1864000,1864008,1864036,1864075,1864076,1864083,1864088,1864101,1864126,1864127,1864128,1864130,1864139,1864143,1864146,1864148,1864166,1864182,1864183,1864184,1864188,1864199,1864200,1864201,1864203,1864204,1864205,1864206,1864207,1864208,1864209,1864210,1864211,1864212,1864213,1864215,1864216,1864217,1